# <h2 align=center> PROYECTO INDIVIDUAL Nº1 </h2>
# <h1 align=center>Recomendacion de Películas</h1>
        @autor: Lesmen Gerardo García Pernia

## Objetivos:

+ Crear un Modelo de Sistema de Recomendacion utilizando las caracteristicas de las películas para encontrar peliculas similares a las que ha visto un determinado usuario y le han gustado.

Estaremos utilizando un Sistema de Recomendacion Basado en el Analisis de Contenido <Br>
El objetivo de este tipo de modelo es analizar y seleccionar desde una gran Base de Datos, productos que sean similares a las caracteristicas de preferencia de un usuario. *"Seleccionar productos similares a aquellos que te gustan"*

Información adicioal:

https://www.futurespace.es/sistemas-de-recomendacion-de-contenidos-adivina-que-piensan-tus-clientes/

https://learn.microsoft.com/es-es/azure/architecture/solution-ideas/articles/build-content-based-recommendation-system-using-recommender


### Librerias Utilizadas:
fastapi==0.93.0<Br>
pandas==1.3.5<Br>
pip==23.2.1<Br>
typing-extensions==4.5.0<Br>
uvicorn==0.20.0<Br>
scikit-learn==1.0.2<Br>
regex==2023.5.5<Br>
sklearn==0.0.post5<Br>
matplotlib==3.7.1<Br>
seaborn=0.12.2<Br>
numpy=1.24.3<Br>

In [472]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

Fuente de datos

+ [Dataset](https://drive.google.com/drive/folders/1mfUVyP3jS-UMdKHERknkQ4gaCRCO2e1v): <Br>
Dos (2) Archivos de datos: movies_dataset.csv y credits.csv
+ [Diccionario de datos](https://docs.google.com/spreadsheets/d/1QkHH5er-74Bpk122tJxy_0D49pJMIwKLurByOfmxzho/edit#gid=0): <br/>
Diccionario con algunas descripciones de las columnas disponibles en el dataset.


# 1) Extracción Transformación y Carga (ETL)

Dataset Originales de datos: movies_dataset.csv y credits.csv

In [473]:
### Datasets Originales de Datos. 
df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')      # Pesa  34 MB
df_credits = pd.read_csv('../DataSetsPI/credits.csv')            # Pesa 186 MB


C:\Users\lesme\AppData\Local\Temp\ipykernel_2084\1988553726.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('../DataSetsPI/movies_dataset.csv')      # Pesa  34 MB


## 1.1) Revision inicial de los datos de "df_movies"

### 1.1.1) Requerimientos Iniciales del Proyecto:

In [474]:
### Visualizar el dataSets de movies:
df_movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Dimensiones de los Datasets

In [477]:
print("Total de Registros del DataSets de df_movies = ",df_movies.shape[0])
print("Total de Columnas del DataSets de df_movies = ",df_movies.shape[1])
print("Total de Registros del DataSets de df_credits = ",df_credits.shape[0])
print("Total de Columnas del DataSets de df_credits = ",df_credits.shape[1])

Total de Registros del DataSets de df_movies =  45466
Total de Columnas del DataSets de df_movies =  24
Total de Registros del DataSets de df_credits =  45476
Total de Columnas del DataSets de df_credits =  3


In [ ]:
### Visualizar los tipos de datos de las columnas y cantidad total:
print("Tipos de Datos de cada Columna del Dataset: 'df_movies' ")
print("="*54)
df_movies.info()

In [ ]:
### Visualizar los datos númericos:
print("Visualizando Estadisticos Principales de los datos númericos del Dataset: 'df_movies' ")
print("="*84)
df_movies.describe()

In [ ]:
### Valores nulos en cada columna de "df_movies"
print("Valores Nulos del Dataset: 'df_movies' ")
print("="*37)
df_movies.isna().sum()

In [ ]:
### Total de valores nulos en "df_movies"
print("Total de valores nulos en 'df_movies'= ", df_movies.isna().sum().sum())

### Se procede a realizar los requerimiento iniciales del proyecto.

### 1.1.1.1) Requerimiento del Proyecto, Nro.: 1

+ Algunos campos, como **`belongs_to_collection`**, **`production_companies`** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder  y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [ ]:
df_movies[['belongs_to_collection','production_companies']]

In [ ]:
### Visualizando el 3er registro del Dataset para visualizar sus datos
df_movies.iloc[2]

Efectivamen el datasets de 'df_movies' tiene varias columnas anidadas. <Br>Se procede a su visualizacion en detalle

### A) Visualización de las Columnas Anidadas del Datasets 'df_movies'

1) Visualizando la columna: *`belongs_to_collection`*

In [ ]:
### Visualizando la cadena en forma de diccionario de la columna "belongs_to_collection", de un determinado registro. 

fila_indice = 4  # Indice del registro o fila que se quiere visualizar.
Columna = 'belongs_to_collection'
CadenaDeDiccionario = df_movies.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(CadenaDeDiccionario,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

### Convertir la cadena de caracteres en un diccionario usando eval()
DiccionarioDeDatos = eval(CadenaDeDiccionario)

### Ciclo for par avisualizar por clave valor, emulando un diccionario a partir de una cadena de caracteres
for key, value in DiccionarioDeDatos.items():
    print(f"{key}: {value}")
 
print("="*100)    
print(f"La columna '{Columna}', se refiere a la franquicia que realizó la película")
print("="*100)     


De la columna "belongs_to_collection" nos interesa extraer el nombre de la Franquicia. Osea extraer la clave: name y su respectivo valor.

2) Visualizando la columna: *`genres`*

In [ ]:
### Visualizando la lista de diccionario de la columna "genres" de un determinado registro. 

fila_indice = 3  # Indice del registro a visualizar.
Columna = 'genres'

CadenaDeDiccionario = df_movies.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(CadenaDeDiccionario,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_movies.at[fila_indice, Columna]   # Toma la cedana de caracteres en forma de lista de diccionarios
lista = ast.literal_eval(lista_str)             # Convierte la cadena en dormato de lista.
N = len(lista)                                 # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:                 # For para imprimir todos los diccionarios de la lista.
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("--------------------")
    
print("="*100)    
print(f"La columna '{Columna}', se refiere a la franquicia que realizó la película. (Dato relevante en el proyecto)")
print("="*100)

De la columna "genres", nos interesa extraer todos los Generos de la Película. En esta caso la película podria tener varios generos. Por lo tanto debemos extraer todos sus claves valor de: name.

3) Visualizando la columna: *`production_companies`*

In [ ]:
### Visualizando la lista de diccionario de la columna "production_companies" de un determinado registro. 

fila_indice = 5  # Indice del registro o fila que se quiere visualizar.
Columna = 'production_companies'

CadenaDeDiccionario = df_movies.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(CadenaDeDiccionario,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_movies.at[fila_indice, Columna]
lista = ast.literal_eval(lista_str)
N = len(lista) # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("--------------------")
    
print("="*100)    
print(f"La columna '{Columna}', se refiere a la franquicia que realizó la película")
print("="*100)

De la columna "production_companies" nos interesa extraer todos las Compañias de Produccion de la Película. En esta caso la película podria tener varias compañias de producción. Por lo tanto debemos extraer todos sus claves valor de: name.

4) Visualizando la columna: *`production_countries`*

In [ ]:
### Visualizando la lista de diccionario de la columna "production_countries" de un determinado registro. 

fila_indice = 3500  # Indice del registro o fila que se quiere visualizar.
Columna = 'production_countries'

CadenaDeDiccionario = df_movies.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(CadenaDeDiccionario,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_movies.at[fila_indice, Columna]
lista = ast.literal_eval(lista_str)
N = len(lista) # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("--------------------")
    
print("="*100)    
print(f"La columna '{Columna}', se refiere a la franquicia que realizó la película")
print("="*100)

De la columna "production_countries" nos interesa extraer todos las Paises de Produccion de la Película. En esta caso la película podria tener varias paises de producción. Por lo tanto debemos extraer todos sus claves valor de: name.

5) Visualizando la columna: *`spoken_languages`*

In [ ]:
### Visualizando la lista de diccionario de la columna "genres" de un determinado registro. 
fila_indice = 2  # Indice del registro o fila que se quiere visualizar.
Columna = 'spoken_languages'

CadenaDeDiccionario = df_movies.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(CadenaDeDiccionario,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_movies.at[fila_indice, Columna]
lista = ast.literal_eval(lista_str)
N = len(lista) # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("--------------------")
    
print("="*100)    
print(f"La columna '{Columna}', se refiere a la franquicia que realizó la película")
print("="*100)

De la columna "spoken_languages" nos interesa extraer todos los Idiomas que se hablan en la película. <Br>En esta caso en la película podrian hablarse varios idiomas. Por lo tanto debemos extraer todos sus claves valor de: name.

In [ ]:
### Visualizando todas las columnas anidadas de un determinado registro o fila.
for i in (1,3,12,13,17):
    print(df_movies.iloc[2][i])

### B) Funciones para Desanidar las Columnas Anidadas del Datasets 'df_movies'

Se procede a desanidar o separar el valor de la clave-valor: en las siguientes columnas:

<ul>
<li> 1) belongs_to_collection (Diccionario).</li>
<li> 2) genres (Lista de Diccionario). </li>
<li> 3) production_companies (Lista de Diccionario).</li>
<li> 4) production_countries (Lista de Diccionario).</li>
<li> 5) spoken_languages (Lista de Diccionario).</li>
</ul>

Se procede a crear dos Funciones para Desanidar estas 5 columnas anidadas. La primera función ***(`Extraer_Valor_Dicc`)***, para extraer de la columna 'belongs_to_collection' sus campos en forma de diccionarios. Y la segunda función ***(`Extraer_Valor_List`)***, para extraer del resto de las columnas que contienen sus campos como una lista de diccionarios.

1 ) Funcion: `Extraer_Valor_Dicc`:<Br>
*Funcion para evaluar la columnas string pero en forma de diccionarios y extraer el valor asociado a la claves claves- valor

In [ ]:
# Función para extraer el valor de la clave 'name', de la columna diccionario: 
def Extraer_Valor_Dicc(value):
    try:
        diccionario = ast.literal_eval(value)  #Evaluamos cada atributo como un diccionario de Python
        if isinstance(diccionario, dict): #Si el valor es un diccionario, extrae el valor asociado a la clave 'name', para luego retornarlo
            return diccionario.get('name', '')
    except (ValueError, SyntaxError): #Si no es posible evaluarlo como un diccionario, capturamos las excepciones (ValueError o SyntaxError) y devolvemos una cadena vacía.
        pass
    return ''  # En caso de no ser un diccionario, se devuelve una cadena vacía.

2  ) Funcion: `Extraer_Valor_Lista`:<Br>
*Funcion para evaluar la columnas string pero en forma de listas de diccionarios y extraer el valor asociado a la claves claves- valor

In [ ]:
# Función para extraer el valor de la clave 'name', de la columna lista de diccionarios: genres
def Extraer_Valor_List(value):
    try:
        lista_de_diccionarios = ast.literal_eval(value) ##Evaluamos cada atributo como una lista de diccionario de Python
        if isinstance(lista_de_diccionarios, list): #Si el valor es una lista de diccionario, extrae el valor asociado a la 
                                                    #clave 'name' de cada diccionario y lo concatena en una sola cadena.
            names = [dic.get('name', '') for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return ','.join(names)
    except (ValueError, SyntaxError):
        pass
    return ''  # Si no es posible evaluarlo como una lista de diccionarios, se devuelve una cadena vacía

### C) Desanidando las Funcióes Anidadas del Datasets 'df_movies'

1) Desanidando la Columna **`'belongs_to_collection'`**, por el atributo {name} que representa el nombre de la Franquicia.

In [ ]:
### Desanidando por la clave-valor de: name
### Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Franquicia'

Columna = 'belongs_to_collection'   # Columna a Desanidar
NuevaColumna = 'Franquicia'         # Nueva Columna

print(f"Cantidad de Valores Nulos de la Columna {Columna}, antes de desanidar= {df_movies[Columna].isna().sum()}")

df_movies[NuevaColumna] = df_movies[Columna].apply(Extraer_Valor_Dicc)

### Eliminando la columna: belongs_to_collection
###df_movies.drop(columns=[Columna], inplace=True)

print(f"Cantidad de Valores Nulos de la Columna {NuevaColumna}, despúes de desanidar: {df_movies[NuevaColumna].isna().sum()}")

### Se procede a eliminar los valores nulos
#df_movies.dropna(subset=[NuevaColumna], inplace=True)
#print(f"Eliminando Valores Nulos. Cantidad de Valores Nulos de la Columna {NuevaColumna}: {df_movies[NuevaColumna].isna().sum()}")

df_movies[NuevaColumna]

In [ ]:
df_movies.isna().sum()

Se puede observar que hay muchos valores NaN

2) Desanidando la Columna **`'genres'`**, por el atributo {name} que representa el nombre los Generos de la Películas.

In [ ]:
### Desanidando por la clave-valor de: name
### Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Generos'

Columna = 'genres'   # Columna a Desanidar
NuevaColumna = 'Generos'         # Nueva Columna

print(f"Cantidad de Valores Nulos de la Columna {Columna}, antes de desanidar= {df_movies[Columna].isna().sum()}")

df_movies[NuevaColumna] = df_movies[Columna].apply(Extraer_Valor_List)

print(f"Cantidad de Valores Nulos de la Columna {NuevaColumna}, despúes de desanidar: {df_movies[NuevaColumna].isna().sum()}")

### Eliminando la columna: genres
### df_movies.drop(columns=[Columna], inplace=True)

### Se procede a eliminar los valores nulos
#df_movies.dropna(subset=[NuevaColumna], inplace=True)
#print(f"Eliminando Valores Nulos. Cantidad de Valores Nulos de la Columna {NuevaColumna}: {df_movies[NuevaColumna].isna().sum()}")

df_movies[NuevaColumna]

In [ ]:
### Total de valores nulos en "df_movies"
print("Total de valores nulos en 'df_movies'= ", df_movies.isna().sum().sum())

Los Generos quedan divididos por comas. También se observa valores en blanco.

3) Desanidando la Columna **`'production_companies'`**, por el atributo {name} que representa el nombre de las Compañias Productoras de Películas.

In [ ]:
### Desanidando por la clave-valor de: name
### Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Productores'

Columna = 'production_companies'     # Columna a Desanidar
NuevaColumna = 'Productores'         # Nueva Columna

print(f"Cantidad de Valores Nulos de la Columna {Columna}, antes de desanidar= {df_movies[Columna].isna().sum()}")

df_movies[NuevaColumna] = df_movies[Columna].apply(Extraer_Valor_List)

### Eliminando la columna: genres
### df_movies.drop(columns=[Columna], inplace=True)

print(f"Cantidad de Valores Nulos de la Columna {NuevaColumna}, despúes de desanidar: {df_movies[NuevaColumna].isna().sum()}")

### Se procede a eliminar los valores nulos
#df_movies.dropna(subset=[NuevaColumna], inplace=True)
#print(f"Eliminando Valores Nulos. Cantidad de Valores Nulos de la Columna {NuevaColumna}: {df_movies[NuevaColumna].isna().sum()}")

df_movies[NuevaColumna]

Las compañias quedan separadas por comas. También se observan registros en blanco.

4) Desanidando la Columna **`'production_countries'`**, por el atributo {name} que representa el nombre del País de Producción de la Película.

In [ ]:
### Desanidando por la clave-valor de: name
### Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Paises'

Columna = 'production_countries'     # Columna a Desanidar
NuevaColumna = 'Paises'              # Nueva Columna

print(f"Cantidad de Valores Nulos de la Columna {Columna}, antes de desanidar= {df_movies[Columna].isna().sum()}")

df_movies[NuevaColumna] = df_movies[Columna].apply(Extraer_Valor_List)

### Eliminando la columna: genres
### df_movies.drop(columns=[Columna], inplace=True)

print(f"Cantidad de Valores Nulos de la Columna {NuevaColumna}, despúes de desanidar: {df_movies[NuevaColumna].isna().sum()}")

### Se procede a eliminar los valores nulos
#df_movies.dropna(subset=[NuevaColumna], inplace=True)
#print(f"Eliminando Valores Nulos. Cantidad de Valores Nulos de la Columna {NuevaColumna}: {df_movies[NuevaColumna].isna().sum()}")

df_movies[NuevaColumna]

5) Desanidando la Columna **`'spoken_languages'`**, por el atributo {name} que representa el nombre de los Idiomas que se hablan en la Película.

In [ ]:
### Desanidando por la clave-valor de: name
### Aplicar la función para extraer el valor 'name' y crear la nueva columna 'IdiomasH'

Columna = 'spoken_languages'           # Columna a Desanidar
NuevaColumna = 'IdiomasH'              # Nueva Columna (Idiomas Hablados en la Película)

print(f"Cantidad de Valores Nulos de la Columna {Columna}, antes de desanidar= {df_movies[Columna].isna().sum()}")

df_movies[NuevaColumna] = df_movies[Columna].apply(Extraer_Valor_List)

### Eliminando la columna: genres
### df_movies.drop(columns=[Columna], inplace=True)

print(f"Cantidad de Valores Nulos de la Columna {NuevaColumna}, despúes de desanidar: {df_movies[NuevaColumna].isna().sum()}")

### Se procede a eliminar los valores nulos
#df_movies.dropna(subset=[NuevaColumna], inplace=True)
#print(f"Eliminando Valores Nulos. Cantidad de Valores Nulos de la Columna {NuevaColumna}: {df_movies[NuevaColumna].isna().sum()}")

df_movies[NuevaColumna]

In [ ]:
### Información de Columnas y Filas del DataSets
print("Columnas del DataSets de Movies:\n\n",df_movies.columns,"\n")
print("Dimensiones del DataSets de Movies: ",df_movies.shape,"\n")

In [ ]:
df_movies.isna().sum()

### 1.1.1.2) Requerimiento del Proyecto, Nro. 2

+ Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.

In [ ]:
print("Cantidad de Valores Nulos de 'revenue':",df_movies['revenue'].isna().sum())
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

In [ ]:
# Se procede a sustituir los 6 valores nulos en la columna "revenue" y "budget", con cero (0).
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['budget'] = df_movies['budget'].fillna(0)

# Otra forma de hacerlo: 
# df_movies2['revenue'].fillna(0,inplace=True)

# Validamos que ya no existan valores nulos:
print("Cantidad de Valores Nulos de 'revenue':",df_movies['revenue'].isna().sum())
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

In [ ]:
### La columna 'budget' esta definida como Objeto, pero es Númerico.
### Convertiendo la columna de tipo objeto a tipo numerico, y los que no sean numericos los convierte en NaN:

df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

### Validano los cambios realizados a númerico y luego si se ha creado algun NaN:
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

In [ ]:
# Se procede a sustituir los 3 valores nuevos por ceros
df_movies['budget'] = df_movies['budget'].fillna(0)

### Validano los cambios realizados a númerico y luego si se ha creado algun NaN:
print("Cantidad de Valores Nulos de 'budget':",df_movies['budget'].isna().sum())

In [ ]:
### Valores Nulos del DataSets
df_movies.isna().sum()

### 1.1.1.3) Requerimiento del Proyecto, Nro. 3

+ Los valores nulos del campo **`release_date`** deben eliminarse.

In [ ]:
print("Total de Valores Nulos de la columna 'release_date'= ",df_movies['release_date'].isna().sum())

Eliminando Valores Nulos en la Columna 'release_date'

In [ ]:
df_movies.dropna(subset=['release_date'], inplace=True)
print("Total de Valores Nulos de la columna 'release_date'= ",df_movies['release_date'].isna().sum())

### 1.1.1.4) Requerimiento del Proyecto, Nro. 4

+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [ ]:
### Validan el tipo de columna 'release_date' y sus datos
print(df_movies['release_date'])
df_movies['release_date'].info()


Se puede observar que la columna 'release_date' es una fecha y esta definido como tipo objeto

Validamos si hay valores nulos

In [ ]:
### Validamos si tiene NaN:
print("Cantidad de Valores Nulos de 'release_date':",df_movies['release_date'].isna().sum())

Convertimos a formato fecha

In [ ]:
### Primero convertimos a datetime la columna 'release_date'
### Y forzamos a convertir en NaN a todos los valores que den error (errors='coerce).

df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

### Validamos nuevamente la cantidad de valores NaN
print("Cantidad de Valores Nulos de 'release_date':",df_movies['release_date'].isna().sum())

In [ ]:
### Validando que se realizó el cambio
df_movies['release_date'].info()

In [ ]:
### Validamos si tiene NaN:
print("Cantidad de Valores Nulos de 'release_date':",df_movies['release_date'].isna().sum())

Se procede a eliminar los 3 nuevos valores nulos que aparecieron después de convertirlos a formato fecha.

In [ ]:
df_movies.dropna(subset=['release_date'], inplace=True)
print("Total de Valores Nulos de la columna 'release_date'= ",df_movies['release_date'].isna().sum())

Se crea la nueva columna del añio

In [ ]:
### Como efectivamente todos los registros de la columna 'release_date' tienen el mismo formato, 
### pasamos a crear la columna de años:

#Creamos la columna 'release_year' que va a contener el año de la columna 'release_date'
df_movies['Anio'] = pd.to_datetime(df_movies['release_date']).dt.year

### Validamos la nueva columna de 'release_year'
print(df_movies[['release_date','Anio']].head(5))
df_movies[['release_date','Anio']].tail(5)

In [ ]:
### Visualizando Valores Nulos
df_movies.isna().sum()

### 1.1.1.5) Requerimiento del Proyecto, Nro. 5

+ Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.

In [ ]:
### Validando que no hay NaN, en ninguna de las dos columnas:
print(df_movies['budget'].isna().sum())
df_movies['revenue'].isna().sum()

In [ ]:
### Validando datos
df_movies[['budget','revenue']].describe()

Se puede observar que ambas columnas tienen valores en ceros (0)

In [ ]:
### Se realiza esta conversion para detectar la existencia de algun dato, con un posible error. (Por si acaso !!!!)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')

Se valida nuevamente si aparecieron valores nulos.

In [ ]:
### Validando que no hay NaN, en ninguna de las dos columnas:
print(df_movies['budget'].isna().sum())
df_movies['revenue'].isna().sum()

In [ ]:
### Creando la columna con la formula del retorno de inversión 'return'
df_movies['return'] = df_movies.apply(lambda x_return: 0 if x_return['budget'] == 0 else (x_return['revenue'] / x_return['budget']), axis=1)

### Visualizando las columnas
print(df_movies[['revenue', 'budget','return']].head(10))
df_movies[['revenue', 'budget','return']].tail(10)

### 1.1.1.6) Requerimiento del Proyecto, Nro. 6

+ Eliminar las siguientes 6 columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`poster_path`** y **`homepage`**.

In [ ]:
### Total de Columnas
print("Total de Columnas=",df_movies.columns.value_counts().sum())

### Cantidad de columnas a eliminar por el Requeriiento Nro.6
print("Nro de Columnas a eliminar el el Requerimienti Nro.6 = 6 ")

### Se procede a eliminar las 6 columnas que no serán utilizadas:
df_movies.drop(columns=['video', 'imdb_id','adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

### Columnas Restantes:
print("Columnas Restantes =",df_movies.columns.value_counts().sum())

In [ ]:
### Información de Columnas y Filas del DataSets
print("Columnas del DataSets de Movies:\n\n",df_movies.columns,"\n")
print("Dimensiones del DataSets de Movies: ",df_movies.shape,"\n")

In [ ]:
### Validando Tipos de Columnas
print(df_movies.info())

In [ ]:
### Visualizando Valores Nulos
df_movies.isna().sum()

### Fin de Requerimientos iniciales del Proyecto

### 

## 1.2) Revision inicial de los datos de "df_credits"

In [ ]:
### Información de Columnas y Filas del DataSets
print("Columnas del DataSets de Movies:\n\n",df_credits.columns,"\n")
print("Dimensiones del DataSets de Movies: ",df_credits.shape,"\n")

In [ ]:
### Visualizando los datos
df_credits.head(2)


### En el  dataset "df_credits" se requiere desanidar sus dos columnas y <Br>filtrar por los atributos relevantes para el proyecto.

### 1.2.1) Visualizando la primera columna: ***`cast`***

### Columna de Reparto de la Pelicula (cast)

Esta columna cast describe información sobre el reparto de una película. <Br>
Cada registro en el DataFrame representa un miembro del reparto de la película y contiene la siguiente información:

+ "cast_id": El ID del reparto al que pertenece el actor o actriz.
+ "character": El personaje que el actor o actriz interpreta en la película.
+ "credit_id": El ID del crédito en la base de datos de la película.
+ "gender": El género del actor o actriz (1 para mujeres, 2 para hombres).
+ "id": El ID del actor o actriz en la base de datos de la película.
+ "name": El nombre del actor o actriz. 
+ "order": El orden en el que aparece el actor o actriz en los créditos de la película.
+ "profile_path": La ruta de acceso al perfil del actor o actriz en la base de datos de la película. <Br>

In [ ]:
### Visualizando la lista de diccionario de la columna "cast" de un determinado registro. 
fila_indice = 1  # Indice del registro o fila que se quiere visualizar.
Columna = 'cast'
DataSets = 'df_credits'

Cadena = df_credits.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(Cadena,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_credits.at[fila_indice, Columna]
lista = ast.literal_eval(lista_str)
N = len(lista) # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("-------------------------------------------------------")
    
print("="*86)    
print(f"La columna '{Columna}', se refiere al Equipo de Reparto de la Película (Cast)")
print("="*86)

Importante detallar en la salida anterior, que el atributo order toma valores naturales positivos incluyendo al cero ( 0,1,2,3); donde el valor 0 (Cero) corresponde al Protagonista, el valor 1 corresponde al Actor Principal o Primer Actor, el valor 2 corresponde al Actor Secundario y asi sucesivamente.

### 1.2.2) Visualizando la primera columna: ***`crew`***

### Columna de Producción de la Pelicula (crew)

La columna "crew" contiene información sobre el equipo de producción de una película. <Br>

Cada registro en el DataFrame contiene la siguiente información:

+   "credit_id": El ID del crédito en la base de datos de la película.
+   "department": El departamento en el que trabaja el miembro del equipo de producción (por ejemplo, dirección, guion, diseño de producción, fotografía, etc.).
+    "gender": El género del miembro del equipo de producción (1 para mujeres, 2 para hombres).
+   "id": El ID del miembro del equipo de producción en la base de datos de la película.
+   "job": El trabajo que realiza el miembro del equipo de producción (por ejemplo, director, guionista, diseñador de producción, director de fotografía, etc.).
+   "name": El nombre del miembro del equipo de producción.
+   "profile_path": La ruta de acceso al perfil del miembro del equipo de producción en la base de datos de la película.

In [ ]:
### Visualizando la lista de diccionario de la columna "crew" de un determinado registro. 
fila_indice = 0  # Indice del registro o fila que se quiere visualizar.
Columna = 'crew'

Cadena = df_credits.at[fila_indice, Columna]

print("1) Visualizando los datos horizontalmente:\n", "="*41)
print(Cadena,"\n")

print("2) Visualizando los datos verticalmente:\n", "="*39)

lista_str = df_credits.at[fila_indice, Columna]
lista = ast.literal_eval(lista_str)
N = len(lista) # Cantidad de diccionarios a mostrar de la lista
for diccionario in lista[:N]:
    for key, value in diccionario.items():
        print(f"{key}: {value}")
    print("-------------------------------------------------------")
    
print("="*86)    
print(f"La columna '{Columna}', se refiere al Equipo de Producción de la Película (Crew)")
print("="*86)

En este caso solo se va a desanidar por name y job ya que es relevante para el Sistema de Recomendación.

### 1.2.3) Función para Desanidar las Columnas: ***`'cast'`*** y ***`'crew'`***<br>
Nombre de la Función: `Extraer_Valor_List_Credits`

In [ ]:
### Se procede desanidadas la columna de 'cast', por las clave valor de: 'character' y 'name'.

claves = ['name']

def Extraer_Valor_List_Credits(cadena, clave):
    try:
        lista_de_diccionarios = ast.literal_eval(cadena)                                               
        if isinstance(lista_de_diccionarios, list):                                      
            lista_encontrada = [str(dic.get(clave, '')) for dic in lista_de_diccionarios if isinstance(dic, dict)]
            return ','.join(lista_encontrada)
    except (ValueError, SyntaxError):
        pass
    return ''  # Si no es posible evaluarlo como una lista de diccionarios, se devuelve una  cadena vacía

### 1.2.3.1) Desanidando la Columna:  ***`cast`***

In [ ]:
### Llamado a la función para desanidar la columna: 'cast'

### Desanidando por la clave: {name}, correspondiente a los actores de la películas.
claves = ['name']

for clave in claves:
    df_credits[f'cast_{clave}'] = df_credits['cast'].apply(Extraer_Valor_List_Credits, clave=clave)

df_credits.head(2)

In [ ]:
### Visualizando la nueva columna 'cast_name', correspondiente a una variable cualitativa ordinal de Actores.
### EL orden es importante.
Columna = 'cast_name'
print("="*86)    
print(f"La columna '{Columna}', correspndiente a una variable cualitativa ordinal de Actores")
print("="*86)
df_credits['cast_name']

Se puede observar que en la nueva columna 'cast_name' se tienen los nombre de los actores ordenados. <Br>El primer nombre corresponde al Protagonista (order = 0), <Br>El siguiente al Primer Actor o Actor Principal (order = 1), <Br>El siguiente al Segundo Actor o Actor Secundario (order = 2), y asi sucesivamente. <Br>Este es un buen ejemplo de una variable donde importa el orden; por lo tanto se denomina Variable Cualitativa Ordinal.

### 1.2.3.2) Desanidando la Columna:  ***`crew`***

In [ ]:
### Llamado a la función para desanidar la columna: 'crew'

### Desanidando por la clave: {job} y {name}, correspondiente la Cargo o rol, y Nombre de la persona que desempeña ese cargo.
claves = ['job', 'name']

for clave in claves:
    df_credits[f'crew_{clave}'] = df_credits['crew'].apply(Extraer_Valor_List_Credits, clave=clave)

df_credits.head(2)

In [ ]:
### Visualizando la nueva columna 'crew_name', correspondiente a una variable cualitativa ordinal por Cargos.
### EL primero corresponde al Director
Columna = 'crew_job'
print("="*86)    
print(f"La columna '{Columna}', correspndiente a una variable cualitativa ordinal de Roles o Cargos")
print("="*86)
df_credits['crew_job']

In [ ]:
### Visualizando la nueva columna 'crew_name', correspondiente a una variable cualitativa ordinal por Cargos.
### EL primero corresponde al Director
df_credits['crew_name']
print("="*86)    
print(f"La columna '{Columna}', correspndiente a una variable cualitativa ordinal de Nombres por Cargo")
print("="*86)
df_credits['crew_name']

In [ ]:
### Validacion de las nuevas columnas
df_credits.columns

In [ ]:
### Informacion de las dimensiones de df_credits
df_credits.info()

No presenta valores Nulos

## 1.3) Eliminación Registros Duplicados de los Datasets:' df_credits y 'df_movies'.

In [ ]:
### Total de Registros duplicados en: "df_movies"
print("Total de Registros Duplicados en: 'df_movies' = ", df_movies[df_movies['id'].duplicated()]['id'].count())

### Total de Registros duplicados en: "df_credits"
print("Total de Registros Duplicados en: 'df_credits' = ", df_credits[df_credits['id'].duplicated()]['id'].count())

Visualizando Registros Duplicados

In [ ]:
### Visualizando y validando los datos de cada registros duplicados de df_credits; ordenados por el codigo de la película:
DataSets = 'df_movies'
print(f"Visualización del Dataset: {DataSets}: \n\n, {df_movies[df_movies.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])}")

In [ ]:
### Visualizando y validando los datos de cada registros duplicados de df_credits; ordenados por el codigo de la película:
DataSets = 'df_credits'
print(f"Visualización del Dataset: {DataSets}: \n\n, {df_credits[df_credits.duplicated(subset=['id'], keep=False)].sort_values(by=['id'])}")

Eliminando Registros Duplicados

In [ ]:
### Eliminando los valores duplicados en df_movies
df_movies.drop_duplicates(subset=['id'], keep='first', inplace=True)
df_movies[df_movies['id'].duplicated()].value_counts().sum()

### Eliminando los valores duplicados en df_credits
df_credits.drop_duplicates(subset=['id'], keep='first', inplace=True)
df_credits[df_credits['id'].duplicated()].value_counts().sum()

In [ ]:
### Total de Registros duplicados en: "df_movies"
print("Total de Registros Duplicados en: 'df_movies' = ", df_movies[df_movies['id'].duplicated()]['id'].count())

### Total de Registros duplicados en: "df_credits"
print("Total de Registros Duplicados en: 'df_credits' = ", df_credits[df_credits['id'].duplicated()]['id'].count())

# 2) Analisís Exploratorios de los Datos (EDA)

## 2.1) Preprocesamiento de Datos:

+   2.1.1) Valores Faltantes
+   2.1.2) Valores Atípicos
+   2.1.3) Escalado / Normalizado
+   2.1.4) Codificación de Variables
+   2.1.5) Seleccion de atributos
+   2.1.6) Ingeniería de Features

### 2.1.1) Valores Faltantes

In [ ]:
### Valores nulos en cada columna de "df_movies"
print("Valores Nulos del Dataset: 'df_movies' ")
print("="*37)
df_movies.isna().sum()

Totalizando Valores Faltantes en cada DataSets:

In [ ]:
#df_movies.isna().sum()
### Total de valores nulos en "df_movies"
print("Total de valores nulos en 'df_movies'= ", df_movies.isna().sum().sum())

#df_movies.isna().sum()
### Total de valores nulos en "df_credits"
print("Total de valores nulos en 'df_credits'= ", df_credits.isna().sum().sum())

El Datasets de 'df_credits' no refleja valores nulos. 

Revisando Columnas con Valores NaN

1) Columna = 'original_language'

In [ ]:
### INformación Detallada de la Columna: 'original_language'
Columna = 'original_language'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")
print(f"Frecuencia del Valor que mas se repite: {df_movies[Columna].describe()[3]}     Porcentaje= {round((df_movies[Columna].describe()[3]/df_movies[Columna].describe()[0])*100,3)} %    Atributo: {df_movies[Columna].describe()[2]}")
df_movies[Columna].describe()

En resumen en la columna 'original_language' tiene el 0,024 % de valores nulos.
En el 70.992 % de los casos, el Idioma predominante es Ingles ('en').<Br>
Por lo tanto se decide imputar los 11 valores nulos con el idioma que tiene la mayor frecuencia = 'en'.

In [ ]:
Columna = 'original_language'
DatosImputar = 'en'

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,5)} %")

print("Realizando la Imputación o reemplazo de los valores..........")
df_movies[Columna].fillna(DatosImputar, inplace=True)

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,5)} %")

2) Columna = 'overview'

In [ ]:
### INformación Detallada de la Columna: 'overview'
Columna = 'overview'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")
print(f"Frecuencia del Valor que mas se repite: {df_movies[Columna].describe()[3]}     Porcentaje= {round((df_movies[Columna].describe()[3]/df_movies[Columna].describe()[0])*100,3)} %    Atributo: {df_movies[Columna].describe()[2]}")
df_movies[Columna].describe()

La información de la columna 'overview' es relevante ya que nos ofrece una caracteristica general de la película. No tener este atributo impacta en la recomendación de la película, en el caso de utilizar esta columna para la dicha recomendación. En resumen, en este momento no es oprtuno eliminar estos valores NaN. Total de valores nulos 941.  (2.075 %) 

3) Columna = 'runtime'

In [ ]:
### Información Detallada de la Columna: 'runtime'
Columna = 'runtime'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")

df_movies[Columna].describe()

En este caso, se procede a imputar con el valor del Cuartil Q3 (107); ya que indica que el 75 % de la duración <Br>de la película dura aproximadamente 107 minutos o menos.<Br> En resumen, se procede a sustituir los 246 valores nulos de la columna 'runtime' por 107 (Tiempo en minutos) ya que la diferencia entre el Q3 y la media (13), es mucho menor que la desviación estandar; qué es la que nos indica la dispersión de los datos.<Br> Estos 246 valores quedarian a menos de una desviación estandar de la media.

In [ ]:
### Información Detallada de la Columna: 'runtime'
Columna = 'runtime'
DatosImputar = 107    # Q3 = Cuartil 3

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")

print("Realizando la Imputación o reemplazo de los valores..........")
df_movies[Columna].fillna(DatosImputar, inplace=True)

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")

4) Columna = 'status'

In [ ]:
### Información Detallada de la Columna: 'status'
Columna = 'status'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")
print(f"Frecuencia del Valor que mas se repite: {df_movies[Columna].describe()[3]}     Porcentaje= {round((df_movies[Columna].describe()[3]/df_movies[Columna].describe()[0])*100,3)} %    Atributo: {df_movies[Columna].describe()[2]}")
df_movies[Columna].describe()

Se puede observar que hay 80 películas (0.176 %) de un total de 45376 con valores nulos en el columa de 'status'. <Br> Es de notar que el 99.205 % de todas las peliculas estan con estatus de 'Release' (Estrenadas). <Br>Es muy probable que para la fecha actual, ya estas películas también esten estrenadas. <Br>Adicionalmente, si una película no ha sido estrenada hasta la fecha actual, entonces el usuario no podrá sugerirla como que le gustó; ya que no ha visto todavía. En este sentido, el Sistema de Recomendación estaría sólo esperando que la vea. <Br>Por tales razones, se procede a imputar este campo por el atributo 'Released'. Cabe destacar, que el error por lo imputación es muy pequeño y que adicionalmente, tiende a disminuir, dado que es solo de tiempo.

In [ ]:
### Información Detallada de la Columna: 'status'
Columna = 'status'
DatosImputar = 'Released'  

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")

print("Realizando la Imputación o reemplazo de los valores..........")
df_movies[Columna].fillna(DatosImputar, inplace=True)

print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}    Porcentaje= {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")

5) Columna = 'tagline'

In [ ]:
### Información Detallada de la Columna: 'tagline'
Columna = 'tagline'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")
print(f"Frecuencia del Valor que mas se repite: {df_movies[Columna].describe()[3]}     Porcentaje= {round((df_movies[Columna].describe()[3]/df_movies[Columna].describe()[0])*100,3)} %    Atributo: {df_movies[Columna].describe()[2]}")
df_movies[Columna].describe()

Esta columna representa menos de la mitad de datos válidos del Dataset. <Br> El 55.04 % de esta columna son datos faltantes, en este caso datos nulos. <Br> Se procede a eliminar esta columna 'tagline'

In [ ]:
### Se procede a eliminar la columna 'tagline':
df_movies.drop(columns=['tagline'], inplace=True)

6) Columna = 'Franquicia'

In [ ]:
### Información Detallada de la Columna: 'Franquicia'
Columna = 'Franquicia'
print(f"Analizando los Valores Nulos en la Columna:   '{Columna}'")
print(f"Total Registros: {df_movies.shape[0]}")
print(f"Total Valores Nulos: {df_movies[Columna].isna().sum()}")
print(f"Porcentaje Valores Nulos: {round((df_movies[Columna].isna().sum()/df_movies.shape[0])*100,3)} %")
print(f"Frecuencia del Valor que mas se repite: {df_movies[Columna].describe()[3]}     Porcentaje= {round((df_movies[Columna].describe()[3]/df_movies[Columna].describe()[0])*100,3)} %    Atributo: {df_movies[Columna].describe()[2]}")
df_movies[Columna].describe()

Se puede observar que la Columna 'Franquicia' tiene el 90.109 % de valores faltantes. Sin embargo, no se procede tomar ninguna acción ya que es una columna clave para responder la consulta de la función Nro.3.

Validando los Cambios Realizados

In [ ]:
### Valores nulos en cada columna de "df_movies"
print("Valores Nulos del Dataset: 'df_movies' ")
print("="*37)
df_movies.isna().sum()

In [ ]:
### Valores nulos en cada columna de "df_movies"
print("Informacion de Variables del Dataset: 'df_movies' ")
print("="*48)
df_movies.info()

Normalizando la columna 'popularity', ya que esta definida como objeto y en realidad es númerica.

In [ ]:
print("Tipo de variable antes del cambio: ",df_movies['popularity'].dtype)
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')
print("Tipo de variable despúes del cambio: ",df_movies['popularity'].dtype)

###   2.1.2) Valores Atípicos

Analizando las Variables Cuantitativas

In [ ]:
### Variables Cualitativas
ColumnasCualitativas = df_movies[['overview', 'spoken_languages', 'status', 'Franquicia', 'Generos','Productores', 'Paises', 'return', 'IdiomasH']]

### Variables Cuantitativas
ColumnasCuantitativas = df_movies[['budget', 'revenue', 'return', 'popularity', 'vote_average', 'vote_count', 'runtime', 'Anio']]

In [ ]:
### Visualizando Estadídtica de las Variables Cuantitativas
df_movies.describe()

Visualizando las Correlaciones de las Variables Cuantitativas

Graficando un diagrama de cajas para visualizar los valores atípicos

In [ ]:
# Selecciona las variables cuantitativas
ColumnasCuantitativas = df_movies[['budget', 'revenue', 'return', 'popularity', 'vote_average', 'vote_count', 'runtime', 'Anio']]

# Calcula la matriz de correlación
correlation_matrix = ColumnasCuantitativas.corr()

# Crea un mapa de calor para visualizar las correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Mapa de Calor de Correlaciones')
plt.show()

No se observa ninguna correlación significativa.<Br> Preguntas a reflexionar: <Br>
1) Una película con una alta popularidad, debería ser recomendada? <Br> 
2) Una película con una alta popularidad, deberia tener un mayor vote_average? Deberian ser directamente proporcionales? (Su Correlación = 0.15. No existe ninguna relación !!!)

In [ ]:
### Grafico de Distribución de Frecuencia
Columna = 'vote_average'

# Crear un histograma y gráfico de densidad para la popularidad sin escala logarítmica
plt.figure(figsize=(8, 6))
sns.histplot(data=df_movies, x= Columna, kde=True)
plt.title('Distribución de Frecuencia')
plt.xlabel(Columna)
plt.ylabel('Frecuencia')
plt.show()

Como podemos observar en la Gráfica de Distribución de Frecuencias, los valores no estan todos en un mismo rango. Por eso, en la siguienta Gráfica utilizamos una función logaritmica para resultar mejor la distribución de la variable 'vote_average'

In [ ]:
# Aplicar escala logarítmica a los valores de vote_average
popularity_log = np.log1p(df_movies['vote_average'])  # np.log1p para evitar el log(0)

# Crear un histograma y gráfico de densidad para la popularidad logaritmizada
plt.figure(figsize=(8, 6))
sns.histplot(data=popularity_log, kde=True)
plt.title('Distribución de Popularidad (Escala Logarítmica)')
plt.xlabel('Log(Popularidad)')
plt.ylabel('Frecuencia')
plt.show()

Definitivamente, de estas variables cuantitativas la "popularidad" y "voto promedio", aunque no tengan ninguna correlación entre ellas, ni tampoco con el resto de variables cuantitativas, podrian ser significativas para la recomendación.

In [ ]:
# Aplicar escala logarítmica a los valores de popularidad
popularity_log = np.log1p(df_movies['popularity'])  # np.log1p para evitar el log(0)

# Crear un histograma y gráfico de densidad para la popularidad logaritmizada
plt.figure(figsize=(8, 6))
sns.histplot(data=popularity_log, kde=True)
plt.title('Distribución de Popularidad (Escala Logarítmica)')
plt.xlabel('Log(Popularidad)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Selecciona las variables cuantitativas
ColumnasCuantitativas = df_movies[['budget', 'revenue', 'return', 'popularity', 'vote_average', 'vote_count', 'runtime', 'Anio']]

# Crea subplots de diagramas de caja individuales para cada variable
plt.figure(figsize=(12, 8))
for i, column in enumerate(ColumnasCuantitativas.columns, 1):
    plt.subplot(2, 4, i)
    sns.boxplot(data=ColumnasCuantitativas[column], color='skyblue')
    
    # Calcula la cantidad de valores atípicos
    q1 = ColumnasCuantitativas[column].quantile(0.25)
    q3 = ColumnasCuantitativas[column].quantile(0.75)
    iqr = q3 - q1
    outliers = ((ColumnasCuantitativas[column] < q1 - 1.5 * iqr) | (ColumnasCuantitativas[column] > q3 + 1.5 * iqr)).sum()
    
    plt.title(f'Diagrama de Caja de {column}\nValores Atípicos: {outliers}')
    plt.xlabel(column)

plt.tight_layout()
plt.savefig("GraficaDispersion.png") 
plt.show()


Las variables con menos valores atípicos son: **El año, el voto promedio y la popularidad.**

Podemos observar que la columna 'popularity' esta definida como objeto, pero es de tipo númerico.  

### 2.1.3) Escalado / Normalizado

Dado que estamos utilizando la Similitud del Coseno, como medida de similitud para hacer la recomendación de las películas, es poco probable realizar algun tipo de escalado y/o normalización de variables. <Br>  La similitud del coseno ya tiene en cuenta la magnitud de las variables y se enfoca en la dirección relativa entre los vectores de características.

Dado que la similitud del coseno se basa en el ángulo entre los vectores y no en la magnitud absoluta; no deberíamos tener problemas de escala que afecten considerablemente a los resultados de la recomendación de la película.

### 2.1.4) Codificación de Variables

La similitud del coseno realiza sus operaciones con vectores numéricos y no requiere una codificación explícita de variables categóricas.

La similitud del coseno calcula la similitud entre dos vectores númericos basándose en el ángulo entre ellos en un espacio n-dimensional. No asigna un valor numérico a las categorías en sí, por lo que no es necesario convertir variables categóricas en variables numéricas codificadas.

### 2.1.5) Selección de Atributos

In [ ]:
df_movies.columns

2.1.5.1) Eliminación de Columnas del Datasets: 'df_movies'

In [ ]:
### Se procede a eliminar las 5 columnas desanidadas y la columna 'release_date':
df_movies.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'release_date' ], inplace=True)

df_movies.columns

2.1.5.2) Selecion de Actores desde el Datasets: 'df_credits', para la recomendacion.

Visualizando la Columna: 'cast_name' 

In [ ]:
### Visualizndo los atributos de la columna 'cast_name' del datasets 'df_credits'
df_credits['cast_name']

Como ya se había mencionado anteriormente, la columna 'cast_name', contiene los nombres de los actores ordenados desde el protagonista, primer actor, segundo actor, tercer actor y así sucesivamente. 

Visualizando la Cantidad de Actores por Películas

In [ ]:
### Cantidad de Actores por Películas

longDeCadaCampo = df_credits['cast_name'].str.split(',').apply(len) # Se hace una division de la cadena 
                                                                    # y se obtienela longitud o tamaño de actores por películas 
print("Cantidad de Actores por Películas\n", longDeCadaCampo[longDeCadaCampo != 3])

Como podemos observar es una cantidad variable por película. <Br>
Dado que los actores son factor muy importante en la recomendación de películas, se procede a extraer como máximo 3 actores por películas, cuales quedaran de la siguiente manera:<Br>
Una primera columna llamada 'Protagonista', con el nombre del Protaginista,<Br>
Una segunda columna llamada 'Actor1', con el nombre del Actor Principal, y<Br>
Una tercera columna llamada 'Actor2', con el nombre del Actor Secundario.<Br>

Función para Crear Columnas por Actores.

In [ ]:
### Realizando la Separacion de los Actores de las Películas

N = 3  # Número de actores a seleccionar de cada película.

# Función para obtener una lista de nombres y completar con "No Hay" si es necesario
def get_names(row):
    names = row.split(',')
    if len(names) >= 1:
        Protagonist = names[0]
    else:
        Protagonist = np.nan
    if len(names) >= 2:
        Actor1 = names[1]
    else:
        Actor1 = np.nan
    if len(names) >= 3:
        Actor2 = names[2]
    else:
        Actor2 = np.nan
    return Protagonist, Actor1, Actor2

# Dividir la columna nombres_personas y obtener nuevas columnas
df_credits[['Protagonista', 'Actor1', 'Actor2']] = pd.DataFrame(df_credits['cast_name'].apply(get_names).tolist())

# Eliminar la columna original nombres_personas si no la necesitas
df_credits.drop(columns=['cast_name'], inplace=True)

# Mostrar el DataFrame resultante
df_credits

Validando Valores Nulos

In [ ]:
### Visualizando Valores Nulos en 'df_credits'
df_credits.isna().sum()

Aparecieron valores nulos en las columnas de Protagonista, Actor1 y Actor2, despúes de separarlos en columnas; los cuales son significativos para el proyecto, ya que la Recomendación de Películas estará basada entre otras, con estas tres variables. Por tal razón, se procede a eliminar estos registros con NaN.

In [ ]:
### Se procede a eliminar los valores nulos de estas tres columnsa ya que serán utilizadas en la recomendación.
df_credits.dropna(subset=['Protagonista', 'Actor1', 'Actor2'], inplace=True)

### Visualizando Valores Nulos en 'df_credits'
df_credits.isna().sum()

2.1.5.3) Selecion del Director desde el Datasets: 'df_credits', para la recomendacion.

In [ ]:
df_credits[['crew_job', 'crew_name']]

Se puede observar que el primer nombre de la columna 'crew_job' representa al Director de la Peliículas. <Br>
Los otros nombres se correspondes a cada uno de los otros roles existentes.

Se procede a crear una sola columna con el nombre del Director.

In [ ]:
# Crear una nueva columna 'Director' con los nombres de los directores
df_credits['Director'] = df_credits['crew_name'].str.split(',').str[0]

### Eliminando columnas de 'crew_job' y 'crew_name'
### Se procede a eliminar las 2 columnas anidadas ya que no la vamos a utilizar mas:
df_credits.drop(columns=['crew_job', 'crew_name'], inplace=True)
df_credits

Validando los Cambios realizados

In [ ]:
### Total de Registros duplicados en: "df_credits"
print("Total Registros Duplicados de 'df_credits' = ",df_credits[df_credits['id'].duplicated()]['id'].count())

### Total de Registros duplicados en: df_movies
print("Total Registros Duplicados de 'df_movies' = ", df_movies[df_movies['id'].duplicated()]['id'].count(),"\n")

### Dimensiones de "df_credits"
print("Dimensiones del Datasets de 'df_credits' = ", df_credits.shape)

### Dimensiones de "df_movies"
print("Dimensiones del Datasets de 'df_movies' = ", df_movies.shape,"\n")
      
### Columnas de "df_credits"
print("Columnas de 'df_credits' = ", df_credits.columns,"\n")

### Columnas de "df_movies"
print("Columnas de 'df_movies' = ", df_movies.columns)     
      

2.1.5.4) Unión de ambos Datasets df_credits, para realizar mejor manipulacion de las variables.

In [ ]:
### Se procede a eliminar las 2 columnas desanidadas del Datasets: 'df_credits':
df_credits.drop(columns=['cast', 'crew'], inplace=True)

df_credits.columns

Validando Informacion de Nulos y tipos de variables ante de unir los dos datasets

In [ ]:
df_movies.isna().sum()

In [ ]:
df_movies.info()

In [ ]:
df_credits.isna().sum()

In [ ]:
df_credits.info()

Se puede observar el codigo de la pelicula son de diferentes tipos en los Datasets. <Br>Se procede tambien a convertirlos en númerico para realizar la unión de ambos Datasets:

In [ ]:
### Se resetean los indices de los dos Dataset, dado las modificaciones que se han realizado.
df_movies.reset_index(drop=True, inplace=True)
df_credits.reset_index(drop=True, inplace=True)

### Se lleva en ambas columnas el tipo de codigo de la película 'id' a ent64 para facilitar el merge:
df_credits['id'] = df_credits['id'].astype('int64')
df_movies['id'] = df_movies['id'].astype('int64')

### Se procede a realizar el merge entre los dos dataframes:
df_movies_JOIN = pd.merge(df_movies, df_credits, on='id')
df_movies_JOIN.columns

In [ ]:
### Validando Valores Nulos
df_movies_JOIN.isna().sum()

2.1.5.5) Selección de Variables para la Recomendación de Películas.

Columnas Selecionadas para la Recomendación

In [ ]:
### Columnas Selecionadas para la Recomendación
ColumnasSR = df_movies_JOIN[['id', 'title', 'Generos', 'Director', 'Protagonista', 'Actor1', 'Actor2', 'popularity', 'Anio','vote_average', 'Franquicia']].copy()
ColumnasSR

La Columna 'Franquicia' está practicamente vacía. Pero, puede ayudar en aquellos casos que exista.

In [ ]:
### Cantidad de Valores Unicos de las Columnas Seleccionadas para la Recomendación
print("Cantidad de Valoque Unicos de cada una de estas Columnas: \n", ColumnasSR.nunique())

### 2.1.6) Ingeníeria de Features

Dado que el cálculo de similitud se basa en las características presentes en los datos y no en una representación numérica específica de ellas, no es necesario realizar transformaciones complejas o ingeniería de características en el sentido tradicional. Las características originales, ya sean cuantitativas o incluso cualitativas, se utilizan directamente en el cálculo de similitud del coseno.

La similitud del coseno opera en el espacio de características original sin requerir codificaciones específicas ni combinaciones de características.

En nuestro caso, la selección de las características relevantes es el factor primordial para nuestra recomendación. Y adicionalmente, en la calidad de los datos que tengamos

# 3) Guardando el DataSets resultante para luego subirlo a GitHub.

### En resumen, estaremos trabajando con un DataSets de 40666 Registros y 22 Columnas

In [ ]:
### Dimensiones del DataSets
df_movies_JOIN.shape

### Cargando el nuevo Dataset resultante para realizar pruebas locales

In [ ]:
### Realizando la carga en disco para subir el nuevo DataSets de Datos: df_movies_JOIN
df_movies_JOIN.to_csv('DataSets/df_movies_JOIN.csv', sep=',')

### 

# 4) Funciones APis

### 

### 4.1) Funciones Requeridas

### Función Nro. 1 <Br>
'''Ingresas el idioma, retornando la cantidad de peliculas producidas en el mismo'''

In [ ]:
df_movies_JOIN['original_language'].nunique()

In [ ]:
### FUNCION Nro. 1
#@app.get('/peliculas_idioma/{idioma}')
def peliculas_idioma(idioma:str):
    '''Ingresas el idioma, retornando la cantidad de peliculas producidas en el mismo'''
    try:
        idioma = idioma.lower()
        mask = df_movies_JOIN[df_movies_JOIN['original_language'].str.lower() == idioma]
        
        if mask.empty:
            {'Error': 'Idioma no encontrado. Intente de nuevo...!!! '}    # Si la mascara esta vacía, entonces no encontro ningun idioma.
        
        respuesta = mask.shape[0]   # Si la mascara no esta vacís, entonces registra la cantidad de registros o filas = Total de Películas.         

        return {'idioma':idioma, 'cantidad':respuesta}   # Retorna el resultado

    except Exception as e:
        return {'error': f'Ocurrió un error: {e}'}

### Probando la funcón::
Idioma = 'es'
Respuesta = peliculas_idioma(Idioma)
print(Respuesta) 

### Función Nro. 2 <Br>
'''Ingresas la pelicula, retornando la duracion y el año'''

In [ ]:
### FUNCION Nro. 2
#@app.get('/peliculas_duracion/{pelicula}')   
def peliculas_duracion(pelicula:str):
    '''Ingresas la pelicula, retornando la duracion y el año'''
    try:
        pelicula = pelicula.lower()
        datos_pelicula = df_movies_JOIN[df_movies_JOIN['title'].str.lower() == pelicula]  # Si no lo encuentra se convierte en un Dataframe vacío
        
        if datos_pelicula.empty:
            return {'error': 'Película no encontrada en el DataFrame.'}
        
        duracion = int(datos_pelicula['runtime'].iloc[0])   # Extrae la primera fila de la columna 'runtime' = Duración
        anio = int(datos_pelicula['Anio'].iloc[0])          # Extrae la primera fila de la columna 'Anio' = Anio

        return {'pelicula':pelicula, 'duracion':duracion, 'anio':anio}

    except Exception as e:
        return {'Error': f'Por favor, corrija este error: {e}'}


### Llamando a la función:
Pelicula = 'Jumanji'
Respuesta = peliculas_duracion(Pelicula)
print(Respuesta)  

### Función Nro. 3 <Br>
'''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''

In [ ]:
### FUNCION Nro. 3
#@app.get('/franquicia/{franquicia}')
def franquicia(franquicia:str):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    try:
        franquicia = franquicia.lower()
        datos_franquicia = df_movies_JOIN[df_movies_JOIN['Franquicia'].str.lower() == franquicia]
        
        if datos_franquicia.empty:
            return {'error': 'Franquicia no encontrada en el DataFrame.'}
        
        cantidad = int(datos_franquicia['Franquicia'].shape[0])
        ganancia_total = float(datos_franquicia['revenue'].sum())
        
        if cantidad == 0:
            ganancia_promedio = 0
        else:
            ganancia_promedio = float(ganancia_total/cantidad)
        
        return {'franquicia':franquicia, 'cantidad':cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}
        
    except Exception as e:
        return {'Error': f'Por favor, corrija este error: {e}'}
### Llamando a la función:
franquicia1 = 'Toy Story Collection'
Respuesta = franquicia(franquicia1)
print(Respuesta) 

### Función Nro. 4 <Br>
'''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''

In [ ]:
### FUNCION Nro. 4 
#@app.get('/peliculas_pais/{pais}')
def peliculas_pais(pais:str):
    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''
    try:
        pais = pais.lower()
        datos_pais = df_movies_JOIN[df_movies_JOIN['Paises'].str.lower() == pais]
        
        if datos_pais.empty:
            return {'error': 'País no encontrado en el DataFrame.'}
        
        cantidad = int(datos_pais['Paises'].shape[0]) 
        
        return {'pais':pais, 'cantidad':cantidad}  
  
    except Exception as e:
        return {'Error': f'Por favor, corrija este error: {e}'} 


### Llamando a la función:
Pais = 'Spain'
Respuesta = peliculas_pais(Pais)
print(Respuesta) 

### Función Nro. 5 <Br>
'''Ingresas la productora, entregandote el revenue total y la cantidad de peliculas que realizo '''

In [ ]:
### FUNCION Nro. 5
#@app.get('/productoras_exitosas/{productora}')
def productoras_exitosas(productora:str):
    '''Ingresas la productora, entregandote el revenue total y la cantidad de peliculas que realizo '''
    try:
        productora = productora.lower() 
        datos_productora = df_movies_JOIN[df_movies_JOIN['Productores'].str.lower().str.contains(productora)]
        
        if datos_productora.empty:
            return {'error': 'Productota no encontrada en el DataFrame.'}
                  
        revenue_total = float(datos_productora['revenue'].sum())
        cantidad = int(datos_productora['revenue'].shape[0])
        
        return {'productora':productora, 'revenue_total': revenue_total,'cantidad':cantidad}
    
    except Exception as e:
        return {'Error': f'Por favor, corrija este error: {e}'}
# Aplicar la función para extraer el valor 'name' y crear la nueva columna 'Collection'

productora = 'Touchstone Pictures'
Respuesta = productoras_exitosas(productora)
print(Respuesta)

### Función Nro. 6 <Br>
''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. <Br>
Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma. En formato Lista '''

''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. <Br>
Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma. En formato Lista '''

In [ ]:
### FUNCION Nro. 6
#@app.get('/get_director/{nombre_director}')
def get_director(nombre_director:str):
    try:
        nombre_director = nombre_director.lower()
        
        datos_director = df_movies_JOIN[df_movies_JOIN['Director'].str.lower().str.contains(nombre_director)]    # #Hacemos una lista de ocurrencia de Directores en un DF temporal
      
        if datos_director.empty:
            return {'error': 'País no encontrada en el DataFrame.'}
        
        # Peliculas del Director con sus respectivas variables       
        Peliculas_del_Director = datos_director[['title', 'Anio', 'revenue', 'budget']]
        
        # Retorno del exito 
        retorno_total_director = float(Peliculas_del_Director['revenue'].sum() / Peliculas_del_Director['budget'].sum())
        
        # Convertir el DataFrame en una lista de diccionarios
        Lista_De_Dicc = Peliculas_del_Director.to_dict('records')
        
        return {'director':nombre_director, 'retorno_total_director':retorno_total_director, 'peliculas':Lista_De_Dicc}
        
    except Exception as e:
        return {'Error': f'Por favor, corrija este error: {e}'} 

### Llamando a la función:
NombreDelDirector = 'John Lasseter'
#NombreDelDirector = 'Larry J'   # No es director######################
#NombreDelDirector = 'Joe Johnston'
Respuesta = get_director(NombreDelDirector)
print(Respuesta)  

### 4.2) Función de Recomendación de 5 Películas

Esta función está utilizando la Simulitud del Coseno para encontrar las películas semilares !!!

In [ ]:
# ML
#@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    try:
        # Convertir el título ingresado por el usuario a minúsculas
        titulo = titulo.lower()
        
        # Buscar el título en minúsculas en el DataFrame
        datos_titulo = df_movies_JOIN.loc[df_movies_JOIN['title'].str.lower() == titulo]
        
        if datos_titulo.empty:   # Si no exite la pelicula envia mensaje de error
            return {'error': 'Película no encontrada en el DataFrame.'}
            
        movie_id = df_movies_JOIN.loc[df_movies_JOIN['title'].str.lower()  == titulo, 'id'].iloc[0]
        # Obteneiendo las características de las siguientes variables predictoras del Dataset
        genre_features = df_movies_JOIN['Generos']
        director_features = df_movies_JOIN['Director']
        protagonist_features = df_movies_JOIN['Protagonista']
        actor1_features = df_movies_JOIN['Actor1']
        actor2_features = df_movies_JOIN['Actor2']
        popularity_features = df_movies_JOIN['popularity'].fillna(0).astype(str)  # Convertir a cadena y llenar valores NaN con '0'
        anio_features = df_movies_JOIN['Anio'].fillna(0).astype(str)  # Convertir a cadena y llenar valores NaN con '0'
        vote_average_features = df_movies_JOIN['vote_average'].fillna(0).astype(str)  # Convertir a cadena y llenar valores NaN con '0'
        franquicia_features = df_movies_JOIN['Franquicia']
        
        # Concatenar todas las características
        all_features = franquicia_features + ' ' + genre_features + ' ' + director_features + ' ' + protagonist_features + ' ' + actor1_features + ' ' + actor2_features + ' ' + popularity_features + ' ' + anio_features  + ' ' + vote_average_features

        # Crear un objeto CountVectorizer para convertir las características en vectores
        vectorizer = CountVectorizer(analyzer='word', lowercase=True, token_pattern=r'\w+')

        # Obtener la matriz de documentos término-frecuencia (DTM) a partir de las características
        all_features_matrix = vectorizer.fit_transform(all_features)

        # Obtener las características de la película que le gustó al usuario
        movie_features = df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Franquicia'].iloc[0] + ' ' + df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Generos'].iloc[0] + ' ' + df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Director'].iloc[0] + ' ' + df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Protagonista'].iloc[0] + ' ' + df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Actor1'].iloc[0] + ' ' + df_movies_JOIN.loc[df_movies_JOIN['id'] == movie_id, 'Actor2'].iloc[0] + ' ' + popularity_features.iloc[0] + ' ' + anio_features.iloc[0] + ' ' + vote_average_features.iloc[0]
        movie_features_matrix = vectorizer.transform([movie_features])

        # Calcular la similitud del coseno entre la película que le gustó al usuario y todas las demás películas
        similarities = cosine_similarity(movie_features_matrix, all_features_matrix)

        # Obtener los índices de las películas más similares
        similar_indices = similarities.argsort()[0][-6:-1]

        # Obtener los títulos de las películas más similares para la recomendación
        recomendacion = df_movies_JOIN.loc[similar_indices, 'title']
        
        recomendacion = recomendacion.tolist()
        
        return {'Películas Recomendadas': recomendacion}
        
    except IndexError:
        return {'Error': 'Película no encontrada. Intente de nuevo...!!! '}
    
    except Exception as e:
        return {'error': f'Ocurrió un error: {e}'}


titulo = 'toy story'
Recomendacion = recomendacion(titulo) 
print(Recomendacion)

# 5) Finalización del Proyecto

### 

# Muchas gracias, por su atención.!!!